<a href="https://colab.research.google.com/github/jmtcabili/CUDA-Project-G1/blob/main/CUDA_Programming_Project_Group_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Group 1:
1. Johan Marlo T. Cabili
2. Emma Celine Conception R. Cacatian
3. Ashantie Louize B. Demanalata
4. Geo Brian P. Hilomen

# Deep Dive: CUDA Programming Project

Checking the GPU being utilized

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Feb 21 06:33:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## C Kernel

In [ ]:
%%writefile C_1DCONV.c

#include <stdio.h>
#include <stdlib.h>
#include <time.h>

void Conv1D(size_t n, float* vec, float* out){
  //tama ba na until n-2?
  for (int i = 0; i < n-2; i++){
    out[i] = (vec[i] + vec[i+1] + vec[i+2])/3.0f;
  }
}

int main(){
  const size_t ARRAY_SIZE = 1<<28;
  const size_t ARRAY_BYTES = ARRAY_SIZE * (sizeof(float));
  const size_t loops = 30;

  float* vec, *out;
  vec = (float*)malloc(ARRAY_BYTES);
  out = (float*)malloc(ARRAY_BYTES);
  clock_t start, end;

  for (int i = 0; i < ARRAY_SIZE; i++){
    vec[i] = (float)(i%3+1);
    //range of values 1-3
  }

  Conv1D(ARRAY_SIZE, vec, out);
  double elapse, time_taken;
  elapse = 0.0f;
  for (int i=0; i<loops; i++){
    start = clock();
    Conv1D(ARRAY_SIZE,vec, out);
    end = clock();
    time_taken = ((double)(end-start))*1E3/CLOCKS_PER_SEC;
    elapse = elapse + time_taken;
  }
  printf("Function (in C) average time for %lu loops is %f milliseconds for array size %lu \n", loops, elapse / loops, ARRAY_SIZE);
	printf("---Correctness Check: C---\n");

  //error checking
  size_t err_count = 0;
  for (int i = 0; i<ARRAY_SIZE-2; i++){
    if (out[i] != (vec[i]+vec[i+1]+vec[i+2])/3.0f){
      err_count+=1;
    }
  }

  printf("First 10 elements: \n");
  for (int i = 0; i < 10; i++){
    printf("%.2f\n", out[i]);
  }
  printf("...\n...\n...\n");

  printf("Last 10 elements: \n");
  for (int i = ARRAY_SIZE-1; i >ARRAY_SIZE-11; i--){
    printf("%.2f\n", out[i]);
  }

  printf("Error count (C Program): %lu\n", err_count);
  //expected output: all cells should be equal to 2.0

  free(vec);
  free(out);
  return 0;

}

Overwriting C_1DCONV.c


In [ ]:
%%shell
gcc C_1DCONV.c -o C_1DConv


In [ ]:
%%shell
./C_1DConv

Function (in C) average time for 30 loops is 1069.611633 milliseconds for array size 268435456 
---Correctness Check: C---
First 10 elements: 
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
...
...
...
Last 10 elements: 
0.00
0.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
Error count (C Program): 0


## Unified Memory

In [ ]:
%%writefile Conv1D.cu

#include <stdio.h>
#include <stdlib.h>

// *** CUDA Conv1D Kernel

__global__
void Conv1D(size_t n, float* in, float* out){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = gridDim.x * blockDim.x;
    for (int i = index; i < n - 2; i += stride) {
        out[i] = (in[i] + in[i+1] + in[i+2])/3.0f;
    }
}

int main(int argc, char** argv) {
    const size_t ARRAY_SIZE = 1<<28;
    const size_t ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
    const size_t loop = 1;

    // Declare array
    float *in, *out;
    cudaMallocManaged(&in, ARRAY_BYTES);
    cudaMallocManaged(&out, ARRAY_BYTES);

    // Get GPU ID
    int device = -1;
    cudaGetDevice(&device);

    // Initialize array
    for (int i = 0; i < ARRAY_SIZE; i++){
        in[i] = (float)(i%3+1);
        //range of values 1-3
    }

    // Setup CUDA kernel
    size_t numThreads = 1024;
    size_t numBlocks = (ARRAY_SIZE + numThreads - 1) / numThreads;

    printf("*** function = Conv1D\n");
    printf("numElements = %lu\n", ARRAY_SIZE);
    printf("numBlocks = %lu, numThreads = %lu\n", numBlocks, numThreads);

    for (size_t i = 0; i < loop; i++)
        Conv1D<<<numBlocks, numThreads>>> (ARRAY_SIZE, in, out);
    // Barrier
    cudaDeviceSynchronize();

    // Error Checking Routine
    size_t err_count = 0;
    for (int i = 0; i<ARRAY_SIZE-2; i++){
        if (out[i] != (in[i] + in[i+1] + in[i+2])/3.0f){
            err_count+=1;
        }
    }
    printf("Error count (CUDA Program): %lu\n", err_count);
    //expected output: all cells should be equal to 2.0


    printf("First 20 elements: \n");
    for (int i = 0; i < 20; i++){
        printf("%.2f\n", out[i]);
    }
    printf("...\n...\n...\n");

    printf("Last 20 elements: \n");
    for (int i = ARRAY_SIZE-20; i < ARRAY_SIZE; i++){
        printf("%.2f\n", out[i]);
    }

    cudaFree(in);
    cudaFree(out);

    return 0;
}

Writing Conv1D.cu


In [ ]:
%%shell
nvcc Conv1D.cu -o Conv1D
nvprof ./Conv1D

==1340== NVPROF is profiling process 1340, command: ./Conv1D
*** function = Conv1D
numElements = 268435456
numBlocks = 262144, numThreads = 1024
Error count (CUDA Program): 0
First 20 elements: 
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
...
...
...
Last 20 elements: 
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
0.00
0.00
==1340== Profiling application: ./Conv1D
==1340== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  978.73ms         1  978.73ms  978.73ms  978.73ms  Conv1D(unsigned long, float*, float*)
      API calls:   68.96%  975.58ms         1  975.58ms  975.58ms  975.58ms  cudaDeviceSynchronize
                   11.56%  163.54ms         2  81.768ms  81.558ms  81.978ms  cudaFree
                   10.76%  152.28ms         2  76.138ms  70.112us  152.21ms  cudaMallocManaged
                    8.64%  12

## Prefetching of Data with Memory Advise

In [ ]:
%%writefile cuda_memadvise_convolution.cu

#include <stdio.h>
#include <stdlib.h>

//CUDA convolution kernel
__global__
void cuda_convolution_withmem(size_t n, float *out, float *in){
    int k;
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;

    for (k = index; k < n-2; k += stride)
       out[k]= (in[k]+ in[k+1]+in[k+2])/3.0f;
}


int main(){
  const size_t ARRAY_SIZE = 1<<28;
  const size_t ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
  const size_t numOfLoops = 30;

  // declare array
    float *in, *out;
    cudaMallocManaged(&in, ARRAY_BYTES);
    cudaMallocManaged(&out, ARRAY_BYTES);

  //get GPU id
    int device = -1;
    cudaGetDevice(&device);

  // memory advise
   cudaMemAdvise(in, ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
   cudaMemAdvise(in, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);

  //"prefetch data" to create CPU page memory
    cudaMemPrefetchAsync(in,ARRAY_BYTES,cudaCpuDeviceId,NULL);
  //"prefetch data" to create GPU page memory
    cudaMemPrefetchAsync(out,ARRAY_BYTES,device,NULL);

  // initialize array
  for (size_t i=0; i<ARRAY_SIZE; i++)
     in[i] = i % 3 + 1.0;

  // prefetch from CPU to GPU
  cudaMemPrefetchAsync(in,ARRAY_BYTES,device,NULL);

  // CUDA kernel
    size_t numThreads = 1024;
    size_t numBlocks = (ARRAY_SIZE + numThreads-1) / numThreads;

  printf("\n***** Function = 1D Convolution implementation in CUDA with MemAdvise\n");
  printf("numElements = %lu\n", ARRAY_SIZE);
  printf("numBlocks = %lu, numThreads = %lu \n",numBlocks, numThreads);
    for (size_t i=0; i<numOfLoops;i++)
      cuda_convolution_withmem <<<numBlocks, numThreads>>> (ARRAY_SIZE,out,in);

  // synchronize GPU with CPU
    cudaDeviceSynchronize();

  // prefetch from GPU to CPU
  cudaMemPrefetchAsync(out,ARRAY_BYTES,cudaCpuDeviceId,NULL);

  // error checking routine
    size_t err_count = 0;
    for (size_t i=0; i<ARRAY_SIZE-2; i++){
      if((in[i]+ in[i+1]+in[i+2])/3.0f != out[i])
        err_count++;
    }


  //Displays First 20 Elements
  printf("First 20 elements: \n");
  for (int i = 0; i < 20; i++){
    printf("%.2f\n", out[i]);
  }
  printf("...\n...\n...\n");


  //Displays Last 20 Elements
  printf("Last 20 elements: \n");
  for (int i = ARRAY_SIZE-20; i < ARRAY_SIZE; i++){
    printf("%.2f\n", out[i]);
  }


  printf("Error count (Prefetch & MemAdvise): %lu\n", err_count);

  //free memory
    cudaFree(in);
    cudaFree(out);
}

Overwriting cuda_memadvise_convolution.cu


In [ ]:
%%shell
nvcc cuda_memadvise_convolution.cu -o cuda_memadvise_convolution

In [ ]:
%%shell
nvprof ./cuda_memadvise_convolution

==1752== NVPROF is profiling process 1752, command: ./cuda_memadvise_convolution

***** Function = 1D Convolution implementation in CUDA with MemAdvise
numElements = 268435456
numBlocks = 262144, numThreads = 1024 
First 20 elements: 
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
...
...
...
Last 20 elements: 
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
0.00
0.00
Error count (Data Initialization as a CUDA Kernel): 0
==1752== Profiling application: ./cuda_memadvise_convolution
==1752== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  309.98ms        30  10.333ms  8.8237ms  14.855ms  cuda_convolution_withmem(unsigned long, float*, float*)
      API calls:   46.95%  435.66ms         4  108.92ms  4.7047ms  237.30ms  cudaMemPrefetchAsync
                   33.39%  309.89ms         1  309.89ms  309.89ms  309.89ms  cu

## Data Initialization in CUDA Kernel

In [ ]:
%%writefile DATAINIT_1DConv.cu

#include <stdio.h>
#include <stdlib.h>


__global__

void initData(size_t n, float* vec){
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;

  for(int i = index;i<n;i += stride)
  {
    vec[i] = (float) (i % 3 + 1.0);
  }
}

__global__
void conv1D_kernel(size_t n, float* vec, float* out){
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;
  for(int i = index;i<n-2;i += stride)
  {
    out[i] = (vec[i] + vec[i+1] + vec[i+2])/3.0f;
  }
}


int main()
{
  //Declare Array Size and Array Bytes
  const size_t ARRAY_SIZE = 1<<28;
  const size_t ARRAY_BYTES = ARRAY_SIZE * (sizeof(float));

  //Declare Amount of Loops
  const int loops = 30;

  //Declare pointers for input and output vectors
  float *in,*out;

  //Allocates the memory for the input and output
  cudaMallocManaged(&in, ARRAY_BYTES);
  cudaMallocManaged(&out, ARRAY_BYTES);

  //Declares the number of blocks and threads
  size_t numThreads = 1024;
  size_t numBlocks = (ARRAY_SIZE + numThreads-1) / numThreads;

  //Initializes Data in the CUDA Kernel
  initData<<<numBlocks,numThreads>>>(ARRAY_SIZE, in);


  printf("*** function = Float Average of 3\n");
  printf("numElements = %lu\n", ARRAY_SIZE);
  printf("numBlocks = %lu, numThreads = %lu \n",numBlocks, numThreads);
  for (size_t i=0; i<loops;i++){

    //Calls the function to do the convolution
    conv1D_kernel<<<numBlocks,numThreads>>>(ARRAY_SIZE, in, out); //asynchronous

  }

  //barrier
    cudaDeviceSynchronize();

//error checking
  size_t err_count = 0;
  for (int i = 0; i<ARRAY_SIZE-2; i++){
    if (out[i] != (in[i]+in[i+1]+in[i+2])/3.0f){
      err_count+=1;
    }
  }

  //Displays First 20 Elements
  printf("First 20 elements: \n");
  for (int i = 0; i < 20; i++){
    printf("%.2f\n", out[i]);
  }
  printf("...\n...\n...\n");


  //Displays Last 20 Elements
  printf("Last 20 elements: \n");
  for (int i = ARRAY_SIZE-20; i < ARRAY_SIZE; i++){
    printf("%.2f\n", out[i]);
  }


  printf("Error count (Data Initialization as a CUDA Kernel): %lu\n", err_count);
  //expected output: all cells should be equal to 2.0


  cudaFree(in);
  cudaFree(out);

  return 0;
}



Overwriting DATAINIT_1DConv.cu


In [ ]:
%%shell
nvcc DATAINIT_1DConv.cu -o DATAINIT_1DConv
nvprof ./DATAINIT_1DConv

==2062== NVPROF is profiling process 2062, command: ./DATAINIT_1DConv
*** function = Float Average of 3
numElements = 268435456
numBlocks = 262144, numThreads = 1024 
First 20 elements: 
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
...
...
...
Last 20 elements: 
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
0.00
0.00
Error count (Data Initialization as a CUDA Kernel): 0
==2062== Profiling application: ./DATAINIT_1DConv
==2062== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   69.64%  484.93ms        30  16.164ms  8.7951ms  229.69ms  conv1D_kernel(unsigned long, float*, float*)
                   30.36%  211.38ms         1  211.38ms  211.38ms  211.38ms  initData(unsigned long, float*)
      API calls:   70.76%  696.14ms         1  696.14ms  696.14ms  696.14ms  cudaDeviceSynchronize
                   17.64%  173.59ms    

## Old Data Transfer Method

In [ ]:
%%writefile OLD_1DConv.cu

#include <stdio.h>

__global__
void Conv1D(size_t n, float* in, float* out){

  int idx = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;

  for (int i = idx; i < n-2; i+=stride){
    out[i] = (in[i] + in[i+1] + in[i+2])/3.0f;
  }
}

int main(){

  const size_t ARRAY_SIZE = 1<<28;
  const size_t ARRAY_BYTES = ARRAY_SIZE * (sizeof(float));
  const int loops = 30;

  //Allocate input and output vectors in host memory
  float* h_in = (float*)malloc(ARRAY_BYTES);
  float* h_out = (float*)malloc(ARRAY_BYTES);

  //initialize values
  for (int i = 0; i < ARRAY_SIZE; i++){
    h_in[i] = (float)(i%3+1);
    //range of values 1-3
  }

  //Allocate vectors in device memory
  float* d_in;
  cudaMalloc(&d_in, ARRAY_BYTES);
  float* d_out;
  cudaMalloc(&d_out, ARRAY_BYTES);

  //Copy input vectors from host memory to device memory
  cudaMemcpy(d_in, h_in, ARRAY_BYTES, cudaMemcpyHostToDevice);

  //invoke kernel
  size_t numThreads = 1024;
  size_t numBlocks = (ARRAY_SIZE + numThreads - 1) / numThreads;

  for (int i = 0; i < loops; i++){
     Conv1D<<<numBlocks, numThreads>>>(ARRAY_SIZE, d_in, d_out);
  }


  //Copy result from device memory to host memory
  cudaMemcpy(h_out, d_out, ARRAY_BYTES, cudaMemcpyDeviceToHost);

  printf("First 10 elements: \n");
  for (int i = 0; i < 10; i++){
    printf("%.2f\n", h_out[i]);
  }
  printf("...\n...\n...\n");

  printf("Last 10 elements: \n");
  for (int i = ARRAY_SIZE-1; i >ARRAY_SIZE-11; i--){
    printf("%.2f\n", h_out[i]);
  }


  cudaFree(d_in);
  cudaFree(d_out);
  free (h_in);
  free (h_out);

  return 0;
}


Overwriting OLD_1DConv.cu


In [ ]:
%%shell
nvcc OLD_1DConv.cu -o OLD_1DConv -arch=sm_75

In [ ]:
%%shell
nvprof ./OLD_1DConv

==851== NVPROF is profiling process 851, command: ./OLD_1DConv
First 10 elements: 
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
...
...
...
Last 10 elements: 
0.00
0.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
2.00
==851== Profiling application: ./OLD_1DConv
==851== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   57.11%  768.28ms         1  768.28ms  768.28ms  768.28ms  [CUDA memcpy DtoH]
                   25.85%  347.72ms        30  11.591ms  8.8083ms  14.774ms  Conv1D(unsigned long, float*, float*)
                   17.04%  229.31ms         1  229.31ms  229.31ms  229.31ms  [CUDA memcpy HtoD]
      API calls:   87.69%  1.34680s         2  673.40ms  229.48ms  1.11732s  cudaMemcpy
                   12.07%  185.30ms         2  92.650ms  143.29us  185.16ms  cudaMalloc
                    0.21%  3.2557ms         2  1.6279ms  654.96us  2.6008ms  cudaFree
                    0.02%  312.61us        30  10.420us  3.3790us

# References
- https://docs.nvidia.com/cuda/cuda-c-programming-guide/
- https://stackoverflow.com/questions/74050524/what-does-the-gpu-architecture-arch-flag-of-nvcc-do
